<a href="https://colab.research.google.com/github/aSafarpoor/Seminar/blob/main/simple_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
from tqdm import tqdm
import random
from collections import Counter
import pandas as pd
import numpy as np
import pickle
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

# just read data (phase 0)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive
%cd MyDrive
%cd Seminar
!ls

/content/drive
/content/drive/MyDrive
/content/drive/MyDrive/Seminar
AMAZON_FASHION.json	  meta_AMAZON_FASHION.json.gz  user_tweets.p
AMAZON_FASHION.json.gz	  meta_data.p
meta_AMAZON_FASHION.json  user_item_matrix.p


In [4]:
with open('user_tweets.p', 'rb') as fp:
    user_tweets = pickle.load(fp)

with open('user_item_matrix.p', 'rb') as fp:
    user_item_matrix = pickle.load(fp)

with open('user_tweets.p', 'rb') as fp:
    user_tweets = pickle.load(fp)

# phase 1: implement simple **Recommender System**

In [5]:
'''
user_tweets
user_item_matrix
user_tweets
'''

'\nuser_tweets\nuser_item_matrix\nuser_tweets\n'

In [6]:
import pandas as pd

In [7]:
df = pd.DataFrame(user_item_matrix)

In [8]:
df

,user,item,rate
0,A1D4G1SNUZWQOT,7106116521,5.0
1,A3DDWDH9PX2YX2,7106116521,2.0
2,A2MWC41EW7XL15,7106116521,4.0
3,A2UH2QQ275NV45,7106116521,2.0
4,A89F3LQADZBS5,7106116521,3.0
...,...,...,...
883631,A1ZSB2Q144UTEY,B01HJHTH5U,5.0
883632,A2CCDV0J5VB6F2,B01HJHTH5U,5.0
883633,A3O90PACS7B61K,B01HJHTH5U,3.0
883634,A2HO94I89U3LNH,B01HJHF97K,3.0


In [9]:
u_list = df.user
i_list = df.item

mappedDict_u = dict(zip(set(u_list), range(len(u_list))))
mappedDict_i = dict(zip(set(i_list), range(len(i_list))))

u = list(map(lambda x: mappedDict_u[x], u_list))
i = list(map(lambda x: mappedDict_i[x], i_list))

df.user = u
df.item = i

In [10]:
df

,user,item,rate
0,1333,66661,5.0
1,605057,66661,2.0
2,300321,66661,4.0
3,35189,66661,2.0
4,605721,66661,3.0
...,...,...,...
883631,645848,109844,5.0
883632,328439,109844,5.0
883633,298477,109844,3.0
883634,408258,133559,3.0


In [36]:
u = list(set(list(df.user)))
u.sort()
it = list(set(list(df.item)))
it.sort()
len(u),len(it),len(df)

(749233, 186189, 883636)

In [18]:
mat = {}
try:
    df = df.reset_index()  # make sure indexes pair with number of rows
    df['item'] = df['item'].apply(int)
    df['user'] = df['user'].apply(int)
    df['rate'] = df['rate'].apply(int)
except:
    print('hi','\n')

c = 0
for index, row in tqdm(df.iterrows()):
    if c>2000:
        break
    c+=1
    if row.user in mat:
        mat[row["user"]][row["item"]]=row["rate"]
    else:
        mat[row["user"]] = np.zeros(len(it))
        mat[row["user"]][row["item"]] = row["rate"]

2001it [00:01, 1850.10it/s]


In [19]:
X = []
for i in mat.values():
    X.append(i)
print(len(X),len(X[0]))
del mat

1994 186189


In [20]:
import numpy as np
from sklearn.decomposition import PCA
from scipy import spatial

In [21]:
import gc
gc.collect()

322

In [24]:
pca = PCA(n_components=20)
pca.fit(X)

PCA(n_components=20)

In [25]:
X_PCAed = pca.transform(X[0:10])
X_PCAed.shape

(10, 20)

In [37]:
lenu = len(u)
leni = len(it)
lenu,leni

(749233, 186189)

In [68]:
ud = {}
for u in tqdm(range(1000)):
    sampleu = df[df['user']==u].item
    X = np.zeros(leni)
    for s in sampleu:
        X[s] = 1
    X_PCAed = pca.transform([X])
    ud[u] = X_PCAed



100%|██████████| 1000/1000 [00:28<00:00, 34.67it/s]


# main part
* before this part, we used pca to create shorted vectors for each user
* now we want to run simple CF here

In [69]:
# choosed_user
cuser = random.randint(0,999)
print("choosed_user: ",cuser)

choosed_user:  209


In [70]:
ud[cuser]

array([[-0.94835869, -0.68600515, -0.01694283,  0.22249707, -0.22251307,
         0.12596752,  0.22721489,  0.18900167,  0.67568211, -0.18862851,
        -0.04318309, -0.0714564 , -0.01695081,  0.03004704,  0.07181269,
         0.00443909, -0.20047754, -0.09366979, -0.15006612, -0.10449938]])

In [71]:
result1 = 1 - spatial.distance.cosine(ud[cuser], ud[cuser])
print(result1)

result = 1 - spatial.distance.cosine(ud[cuser], ud[cuser+i])
print(result)

1.0
1.0


# test model with random numbers

In [72]:
ud = {}
for u in tqdm(range(1000)):
    X = np.random.randint(20, size=leni)
    X_PCAed = pca.transform([X])
    ud[u] = X_PCAed

100%|██████████| 1000/1000 [00:27<00:00, 36.06it/s]


In [80]:
def similarity_comparator(x,y):
    ''' best 100 worst -100'''
    result = 1 - spatial.distance.cosine(x,y)
    return(int(result*100))

In [81]:
result = similarity_comparator(ud[cuser], ud[cuser])
print(result)

result = similarity_comparator(ud[cuser], -1*ud[cuser])
print(result)

result = similarity_comparator(ud[cuser], ud[cuser+1])
print(result)

result = similarity_comparator(ud[cuser], ud[cuser+2])
print(result)

100
-100
29
-5
